## This code is to combine results from various outputs for PGE Data using Year 1 (test data) and Year 2 (test data) to train and Year 3 to predicit

In [1]:
# Commonly used python functions and display settings
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
# Key imports for this code (various ML and Stat Models)
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression

In [3]:
import constants
import helper_methods

## Train Individual Models and Generate Excel for Predictions

In [ ]:
generate_individual_predictions = True

In [5]:
if generate_individual_predictions:
    %run PK_MODEL.py
    %run NG_MODEL.py
    %run NU_MODEL.py
    %run BY_MODEL.py

In [6]:
if generate_individual_predictions:
    %run SA_MODEL.py

## Get data and analyze

In [7]:
# fetch data from the excel file


sa_train_data = pd.concat([pd.read_excel(constants.SA_TEST_1), pd.read_excel(constants.SA_TEST_2)], axis=0, ignore_index=True)
sa_test_data = pd.read_excel(constants.SA_TEST_3)
sa = constants.SA_COL

by_train_data = pd.concat([pd.read_excel(constants.BY_TEST_1), pd.read_excel(constants.BY_TEST_2)], axis=0, ignore_index=True)
by_test_data = pd.read_excel(constants.BY_TEST_3)
by = constants.BY_COL


nu_train_data = pd.concat([pd.read_excel(constants.NU_TEST_1), pd.read_excel(constants.NU_TEST_2)], axis=0, ignore_index=True)
nu_test_data = pd.read_excel(constants.NU_TEST_3)
nu = constants.NU_COL


pk_train_data = pd.concat([pd.read_excel(constants.PK_TEST_1), pd.read_excel(constants.PK_TEST_2)], axis=0, ignore_index=True)
pk_test_data = pd.read_excel(constants.PK_TEST_3)
pk = constants.PK_COL


ng_train_data = pd.concat([pd.read_excel(constants.NG_TEST_1), pd.read_excel(constants.NG_TEST_2)], axis=0, ignore_index=True)
ng_test_data = pd.read_excel(constants.NG_TEST_3)
ng = constants.NG_COL

methods = [pk, sa, nu, by, ng]

allcols = ['Year', 'Month', 'Day', 'Hour', 'Load', 'Site-1 Temp', 'Site-2 Temp',
       'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp', 'Site-1 GHI', 'Site-2 GHI',
       'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI']

cols = ['Year', 'Month', 'Day', 'Hour']

train_data = ng_train_data[allcols + [ng]]
train_data = train_data.merge(pk_train_data[cols+[pk]], on = cols, how = 'left')
train_data = train_data.merge(by_train_data[cols+[by]], on = cols, how = 'left')
train_data = train_data.merge(nu_train_data[cols+[nu]], on = cols, how = 'left')
train_data = train_data.merge(sa_train_data[cols+[sa]], on = cols, how = 'left')

test_data = ng_test_data[allcols + [ng]]
test_data = test_data.merge(pk_test_data[cols+[pk]], on = cols, how = 'left')
test_data = test_data.merge(by_test_data[cols+[by]], on = cols, how = 'left')
test_data = test_data.merge(nu_test_data[cols+[nu]], on = cols, how = 'left')
test_data = test_data.merge(sa_test_data[cols+[sa]], on = cols, how = 'left')

train_data.head()
train_data.tail()

test_data.head()
test_data.tail()

# Finding how many rows of data we have and if there are any NaN values
len(train_data)
len(test_data)
train_data.isna().sum()
test_data.isna().sum()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
0,1,1,1,1,1997,8.00,8.20,5.30,9.40,8.10,0,0,0,0,0,"1,935.31",NaN,"1,931.47","1,905.52",NaN
1,1,1,1,2,1921,8.30,8.60,5.20,8.60,7.10,0,0,0,0,0,"1,860.35",NaN,"1,852.36","2,100.16",NaN
2,1,1,1,3,1861,8.10,8.80,5.10,8.70,6.20,0,0,0,0,0,"1,829.12",NaN,"1,814.53","2,077.88",NaN
3,1,1,1,4,1833,7.60,8.10,4.30,8.50,6.00,0,0,0,0,0,"1,817.96",NaN,"1,815.76","2,062.45",NaN
4,1,1,1,5,1847,7.30,7.50,4.00,8.60,6.90,0,0,0,0,0,"1,905.81",NaN,"1,882.84","2,007.13",NaN


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
17539,2,12,31,20,2545,12.50,11.90,10.30,12.60,11.30,0,0,0,0,0,"2,609.52","2,556.20","2,608.76","2,584.44","2,706.96"
17540,2,12,31,21,2449,12.50,11.90,9.70,12.60,11.20,0,0,0,0,0,"2,509.41","2,444.91","2,529.14","2,489.97","2,529.90"
17541,2,12,31,22,2348,12.50,12.00,9.70,12.70,11.10,0,0,0,0,0,"2,372.92","2,294.96","2,391.28","2,334.18","2,427.21"
17542,2,12,31,23,2229,12.60,12.40,9.50,12.70,11.00,0,0,0,0,0,"2,195.36","2,208.06","2,227.20","2,167.55","2,206.03"
17543,2,12,31,24,2118,12.40,12.10,9.70,12.50,10.60,0,0,0,0,0,"2,072.39","2,008.86","2,063.95","2,058.77","2,036.59"


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,0,0,0,0,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82"
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,0,0,0,0,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92"
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,0,0,0,0,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19"
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,0,0,0,0,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54"
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,0,0,0,0,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05"


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
8755,3,12,31,20,0,14.50,14.10,13.00,15.00,13.80,0,0,0,0,0,"2,621.84","2,488.42","2,567.14","2,571.57","2,621.74"
8756,3,12,31,21,0,14.60,14.30,13.40,14.90,13.90,0,0,0,0,0,"2,536.27","2,359.44","2,468.81","2,484.98","2,501.05"
8757,3,12,31,22,0,14.50,14.30,13.40,14.90,14.00,0,0,0,0,0,"2,389.25","2,253.15","2,330.30","2,356.05","2,435.20"
8758,3,12,31,23,0,14.40,14.30,12.80,14.80,13.90,0,0,0,0,0,"2,223.32","2,193.10","2,180.88","2,215.05","2,254.00"
8759,3,12,31,24,0,14.40,14.30,12.70,14.70,13.70,0,0,0,0,0,"2,096.99","2,049.57","2,043.23","2,116.64","2,089.42"


17544

8760

Year            0
Month           0
Day             0
Hour            0
Load            0
Site-1 Temp     0
Site-2 Temp     0
Site-3 Temp     0
Site-4 Temp     0
Site-5 Temp     0
Site-1 GHI      0
Site-2 GHI      0
Site-3 GHI      0
Site-4 GHI      0
Site-5 GHI      0
Model NG        0
Model PK       48
Model BY        0
Model NU        0
Model SA       24
dtype: int64

Year           0
Month          0
Day            0
Hour           0
Load           0
Site-1 Temp    0
Site-2 Temp    0
Site-3 Temp    0
Site-4 Temp    0
Site-5 Temp    0
Site-1 GHI     0
Site-2 GHI     0
Site-3 GHI     0
Site-4 GHI     0
Site-5 GHI     0
Model NG       0
Model PK       0
Model BY       0
Model NU       0
Model SA       0
dtype: int64

In [8]:
train_data['avg_to_impute'] =  (train_data[ng] + train_data[nu] )/ 2
train_data[pk].fillna(train_data['avg_to_impute'], inplace = True)
train_data[sa].fillna(train_data['avg_to_impute'], inplace = True)
train_data.drop(columns = ['avg_to_impute'], inplace = True)
train_data.head()


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
0,1,1,1,1,1997,8.00,8.20,5.30,9.40,8.10,0,0,0,0,0,"1,935.31","1,920.41","1,931.47","1,905.52","1,920.41"
1,1,1,1,2,1921,8.30,8.60,5.20,8.60,7.10,0,0,0,0,0,"1,860.35","1,980.25","1,852.36","2,100.16","1,980.25"
2,1,1,1,3,1861,8.10,8.80,5.10,8.70,6.20,0,0,0,0,0,"1,829.12","1,953.50","1,814.53","2,077.88","1,953.50"
3,1,1,1,4,1833,7.60,8.10,4.30,8.50,6.00,0,0,0,0,0,"1,817.96","1,940.21","1,815.76","2,062.45","1,940.21"
4,1,1,1,5,1847,7.30,7.50,4.00,8.60,6.90,0,0,0,0,0,"1,905.81","1,956.47","1,882.84","2,007.13","1,956.47"


In [9]:
test_data['avg_to_impute'] =  (test_data[ng] + test_data[nu] ) / 2
test_data[pk].fillna(test_data['avg_to_impute'], inplace = True)
test_data[sa].fillna(test_data['avg_to_impute'], inplace = True)
test_data.drop(columns = ['avg_to_impute'], inplace = True)
test_data.head()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,Site-1 GHI,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,0,0,0,0,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82"
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,0,0,0,0,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92"
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,0,0,0,0,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19"
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,0,0,0,0,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54"
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,0,0,0,0,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05"


## Classify to get best among the 7 methods

In [10]:
# Function to find the best method and its corresponding prediction
def find_best_method(row):
    errors = {method: abs(row[method] - row['Load']) for method in methods}
    best_method = max(errors, key=lambda k: (-errors[k], k))  # Prioritizing later columns in case of ties
    return best_method, row[best_method]

# Apply function to get best method and best prediction
train_data[['best', 'best_pred']] = train_data.apply(lambda row: pd.Series(find_best_method(row)), axis=1)

train_data.head()
train_data.tail()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best,best_pred
0,1,1,1,1,1997,8.00,8.20,5.30,9.40,8.10,...,0,0,0,"1,935.31","1,920.41","1,931.47","1,905.52","1,920.41",Model NG,"1,935.31"
1,1,1,1,2,1921,8.30,8.60,5.20,8.60,7.10,...,0,0,0,"1,860.35","1,980.25","1,852.36","2,100.16","1,980.25",Model SA,"1,980.25"
2,1,1,1,3,1861,8.10,8.80,5.10,8.70,6.20,...,0,0,0,"1,829.12","1,953.50","1,814.53","2,077.88","1,953.50",Model NG,"1,829.12"
3,1,1,1,4,1833,7.60,8.10,4.30,8.50,6.00,...,0,0,0,"1,817.96","1,940.21","1,815.76","2,062.45","1,940.21",Model NG,"1,817.96"
4,1,1,1,5,1847,7.30,7.50,4.00,8.60,6.90,...,0,0,0,"1,905.81","1,956.47","1,882.84","2,007.13","1,956.47",Model BY,"1,882.84"


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best,best_pred
17539,2,12,31,20,2545,12.50,11.90,10.30,12.60,11.30,...,0,0,0,"2,609.52","2,556.20","2,608.76","2,584.44","2,706.96",Model PK,"2,556.20"
17540,2,12,31,21,2449,12.50,11.90,9.70,12.60,11.20,...,0,0,0,"2,509.41","2,444.91","2,529.14","2,489.97","2,529.90",Model PK,"2,444.91"
17541,2,12,31,22,2348,12.50,12.00,9.70,12.70,11.10,...,0,0,0,"2,372.92","2,294.96","2,391.28","2,334.18","2,427.21",Model NU,"2,334.18"
17542,2,12,31,23,2229,12.60,12.40,9.50,12.70,11.00,...,0,0,0,"2,195.36","2,208.06","2,227.20","2,167.55","2,206.03",Model BY,"2,227.20"
17543,2,12,31,24,2118,12.40,12.10,9.70,12.50,10.60,...,0,0,0,"2,072.39","2,008.86","2,063.95","2,058.77","2,036.59",Model NG,"2,072.39"


In [11]:
# Define mapping
class_mapping = {methods[i]: i for i in range(len(methods))}

# Apply mapping to 'best' column in df_train
train_data['best_encoded'] = train_data['best'].map(class_mapping)


In [12]:
# Creating training data dropping columns not needed and also ground truth
X_train = train_data.drop(columns = ['Year', 'Day', 'Load', 'best_pred', 'best', 'best_encoded'] + methods)
y_train = train_data['best_encoded']
X_test = test_data.drop(columns = ['Year', 'Day', 'Load'] + methods)

In [13]:
# Define the XGBoost regressor with specific hyperparameters
model = XGBClassifier(
        n_estimators=400,
        max_depth=5,
        learning_rate=0.05,
        subsample=1.0,
        colsample_bytree=0.6,
        objective='reg:squarederror',
        random_state=42
    )

# Train the model
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [14]:
# Make predictions
# y_preds = gb.predict(X_test)
y_preds = model.predict(X_test)
y_preds

array([0, 0, 1, ..., 3, 0, 0])

In [15]:
test_data['best_encoded'] = y_preds
train_data['best_encoded_fcst'] = model.predict(X_train)
test_data.head()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-2 GHI,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best_encoded
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,...,0,0,0,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82",0
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,...,0,0,0,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92",0
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,...,0,0,0,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19",1
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,...,0,0,0,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54",1
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,...,0,0,0,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05",1


In [16]:
# Reverse mapping
inverse_mapping = {v: k for k, v in class_mapping.items()}

# Apply inverse mapping to predicted classes
test_data['best'] = test_data['best_encoded'].map(inverse_mapping)  # Assuming predictions are stored in 'best_encoded'
train_data['best_fcst'] = train_data['best_encoded_fcst'].map(inverse_mapping) 
test_data.head()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-3 GHI,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best_encoded,best
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,...,0,0,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82",0,Model PK
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,...,0,0,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92",0,Model PK
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,...,0,0,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19",1,Model SA
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,...,0,0,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54",1,Model SA
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,...,0,0,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05",1,Model SA


In [17]:
# Populate 'best_pred' based on the chosen best method
test_data['best_pred'] = test_data.apply(lambda row: row[row['best']], axis=1)
train_data['best_pred_fcst'] = train_data.apply(lambda row: row[row['best_fcst']], axis=1)
test_data.head()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-4 GHI,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best_encoded,best,best_pred
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,...,0,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82",0,Model PK,"1,919.80"
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,...,0,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92",0,Model PK,"1,891.51"
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,...,0,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19",1,Model SA,"1,894.19"
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,...,0,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54",1,Model SA,"1,855.54"
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,...,0,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05",1,Model SA,"1,840.05"


## Regression to get weighted sum of 7 predictions

In [18]:
# Creating training data dropping columns not needed and also ground truth
X_train = train_data[methods] 
y_train = train_data['Load']
X_test = test_data[methods]
y_test = test_data['Load']

In [19]:
model = LinearRegression(fit_intercept = False)
model.fit(X_train, y_train) 

# The following gives the R-square score
model.score(X_train, y_train) 

# This is the coefficient Beta_1 to Beta_m
model.coef_

LinearRegression(fit_intercept=False)

0.8372863421241596

array([0.40639529, 0.25745734, 0.12505091, 0.03891522, 0.1738088 ])

In [20]:
y_pred = model.predict(X_test)

In [21]:
train_data['weighted_pred'] = model.predict(X_train)
test_data['weighted_pred'] = model.predict(X_test)

In [22]:
train_data.head()
test_data.head()

,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Model BY,Model NU,Model SA,best,best_pred,best_encoded,best_encoded_fcst,best_fcst,best_pred_fcst,weighted_pred
0,1,1,1,1,1997,8.00,8.20,5.30,9.40,8.10,...,"1,931.47","1,905.52","1,920.41",Model NG,"1,935.31",4,4,Model NG,"1,935.31","1,924.70"
1,1,1,1,2,1921,8.30,8.60,5.20,8.60,7.10,...,"1,852.36","2,100.16","1,980.25",Model SA,"1,980.25",1,4,Model NG,"1,860.35","1,972.65"
2,1,1,1,3,1861,8.10,8.80,5.10,8.70,6.20,...,"1,814.53","2,077.88","1,953.50",Model NG,"1,829.12",4,4,Model NG,"1,829.12","1,945.21"
3,1,1,1,4,1833,7.60,8.10,4.30,8.50,6.00,...,"1,815.76","2,062.45","1,940.21",Model NG,"1,817.96",4,4,Model NG,"1,817.96","1,932.56"
4,1,1,1,5,1847,7.30,7.50,4.00,8.60,6.90,...,"1,882.84","2,007.13","1,956.47",Model BY,"1,882.84",3,4,Model NG,"1,905.81","1,954.32"


,Year,Month,Day,Hour,Load,Site-1 Temp,Site-2 Temp,Site-3 Temp,Site-4 Temp,Site-5 Temp,...,Site-5 GHI,Model NG,Model PK,Model BY,Model NU,Model SA,best_encoded,best,best_pred,weighted_pred
0,3,1,1,1,0,12.30,11.50,9.30,12.70,10.80,...,0,"1,899.98","1,919.80","1,853.90","1,933.23","2,017.82",0,Model PK,"1,919.80","1,943.83"
1,3,1,1,2,0,12.10,11.10,9.30,12.30,10.60,...,0,"1,824.27","1,891.51","1,796.96","1,841.17","1,934.92",0,Model PK,"1,891.51","1,884.10"
2,3,1,1,3,0,10.80,10.20,9.00,11.80,9.90,...,0,"1,777.84","1,844.64","1,776.43","1,834.77","1,894.19",1,Model SA,"1,894.19","1,844.90"
3,3,1,1,4,0,10.00,10.30,9.30,10.90,8.90,...,0,"1,787.13","1,846.37","1,782.26","1,858.02","1,855.54",1,Model SA,"1,855.54","1,840.40"
4,3,1,1,5,0,11.10,10.30,9.80,10.90,9.60,...,0,"1,860.76","1,912.51","1,837.41","1,938.39","1,840.05",1,Model SA,"1,840.05","1,888.29"


## Residual based method using the best pred so far

In [23]:
# Creating Residual
train_data['Residuals'] = train_data['Load'] - train_data[pk] # old sa, by
# Creating training data dropping columns not needed and also ground truth
X_train = train_data[['Month', 'Hour', 'Site-1 Temp', 'Site-2 Temp',
       'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp', 'Site-1 GHI', 'Site-2 GHI',
       'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI']]
y_train = train_data['Residuals']
X_test = test_data[['Month', 'Hour', 'Site-1 Temp', 'Site-2 Temp',
       'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp', 'Site-1 GHI', 'Site-2 GHI',
       'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI']]

# Define the XGBoost regressor with specific hyperparameters
model = XGBRegressor(
        n_estimators=400,
        max_depth=5,
        learning_rate=0.05,
        subsample=1.0,
        colsample_bytree=0.6,
        objective='reg:squarederror',
        random_state=42
    )
# Train the model
model.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [24]:
# test_data['Residuals'] = gb.predict(X_test)
test_data['Residuals'] = model.predict(X_test)
test_data['Res_pred'] = test_data['Residuals'] + test_data[pk] # old sa, by
train_data['Res_pred'] = model.predict(X_train) + train_data[pk]

## ML-based corrections for best pred so far

In [25]:
# Creating training data dropping columns not needed and also ground truth
X_train = train_data[['Month', 'Hour', 'Site-1 Temp', 'Site-2 Temp',
       'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp', 'Site-1 GHI', 'Site-2 GHI',
       'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI', pk]] # old sa, by
y_train = train_data['Load']
X_test = test_data[['Month', 'Hour', 'Site-1 Temp', 'Site-2 Temp',
       'Site-3 Temp', 'Site-4 Temp', 'Site-5 Temp', 'Site-1 GHI', 'Site-2 GHI',
       'Site-3 GHI', 'Site-4 GHI', 'Site-5 GHI', pk]] # old sa, by

# Define the XGBoost regressor with specific hyperparameters
model = XGBRegressor(
        n_estimators=400,
        max_depth=5,
        learning_rate=0.05,
        subsample=1.0,
        colsample_bytree=0.6,
        objective='reg:squarederror',
        random_state=42
    )
# Train the model
model.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [26]:
# test_data['Adj_pred'] = gb.predict(X_test)
test_data['Adj_pred'] = model.predict(X_test)
train_data['Adj_pred'] = model.predict(X_train)

## Comparing Predictions

In [27]:
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score, mean_squared_error

# List of prediction columns
prediction_columns = methods + ['best_pred', 'weighted_pred', 'Res_pred', 'Adj_pred']
train_pred_columns = methods + ['best_pred_fcst', 'weighted_pred', 'Res_pred', 'Adj_pred']

# Function to compute metrics
def compute_metrics(df, predictions, ground_truth):
    metrics = []
    for col in predictions:
        y_true = df[ground_truth]
        y_pred = df[col]
        mae = mean_absolute_error(y_true, y_pred)
        medae = median_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Mean Absolute Percentage Error
        metrics.append([col, mae, medae, r2, rmse, mape])
    
    return pd.DataFrame(metrics, columns=['Prediction', 'MAE', 'MedAE', 'R-squared', 'RMSE', 'MAPE'])

# Compute and display metrics
train_metrics_df = compute_metrics(train_data, train_pred_columns, 'Load')
print(train_metrics_df)
test_metrics_df = compute_metrics(test_data, prediction_columns, 'Load')
# print(metrics_df)

       Prediction    MAE  MedAE  R-squared   RMSE  MAPE
0        Model PK 134.45 104.11       0.83 181.27  6.42
1        Model SA 140.12 108.92       0.82 186.03  6.76
2        Model NU 139.53 107.21       0.81 190.46  6.65
3        Model BY 139.34 105.75       0.80 194.13  6.62
4        Model NG 137.84 105.39       0.81 190.45  6.55
5  best_pred_fcst 109.43  81.14       0.88 153.66  5.25
6   weighted_pred 131.34 102.17       0.84 176.37  6.31
7        Res_pred 104.44  83.95       0.90 136.13  5.06
8        Adj_pred  92.01  71.54       0.92 122.27  4.43


In [28]:
train_metrics_df.head(12)
helper_methods.generate_results_latex(train_metrics_df)

,Prediction,MAE,MedAE,R-squared,RMSE,MAPE
0,Model PK,134.45,104.11,0.83,181.27,6.42
1,Model SA,140.12,108.92,0.82,186.03,6.76
2,Model NU,139.53,107.21,0.81,190.46,6.65
3,Model BY,139.34,105.75,0.80,194.13,6.62
4,Model NG,137.84,105.39,0.81,190.45,6.55
5,best_pred_fcst,109.43,81.14,0.88,153.66,5.25
6,weighted_pred,131.34,102.17,0.84,176.37,6.31
7,Res_pred,104.44,83.95,0.90,136.13,5.06
8,Adj_pred,92.01,71.54,0.92,122.27,4.43


Model PK & 134.45 & 104.11 & 0.83 & 181.27 & 6.42 \\
Model SA & 140.12 & 108.92 & 0.82 & 186.03 & 6.76 \\
Model NU & 139.53 & 107.21 & 0.81 & 190.46 & 6.65 \\
Model BY & 139.34 & 105.75 & 0.80 & 194.13 & 6.62 \\
Model NG & 137.84 & 105.39 & 0.81 & 190.45 & 6.55 \\
best_pred_fcst & 109.43 & 81.14 & 0.88 & 153.66 & 5.25 \\
weighted_pred & 131.34 & 102.17 & 0.84 & 176.37 & 6.31 \\
Res_pred & 104.44 & 83.95 & 0.90 & 136.13 & 5.06 \\
Adj_pred & 92.01 & 71.54 & 0.92 & 122.27 & 4.43 \\


In [29]:
test_metrics_df.head(12)

,Prediction,MAE,MedAE,R-squared,RMSE,MAPE
0,Model PK,"2,174.90","2,099.43",0.00,"2,216.67",inf
1,Model SA,"2,169.95","2,086.74",0.00,"2,210.07",inf
2,Model NU,"2,171.83","2,095.87",0.00,"2,213.43",inf
3,Model BY,"2,168.96","2,101.07",0.00,"2,209.33",inf
4,Model NG,"2,170.07","2,098.69",0.00,"2,211.04",inf
5,best_pred,"2,173.54","2,098.49",0.00,"2,214.50",inf
6,weighted_pred,"2,175.71","2,100.91",0.00,"2,216.41",inf
7,Res_pred,"2,184.57","2,097.55",0.00,"2,235.75",inf
8,Adj_pred,"2,175.56","2,093.20",0.00,"2,220.10",inf


In [30]:
methods # Used for best_pred and weighted_pred

['Model PK', 'Model SA', 'Model NU', 'Model BY', 'Model NG']

## Generate Final Excell

In [31]:
test_data['Load'] = test_data['best_pred']
test_data[['Load']].to_excel('Results_OrangeOracle.xlsx', index=False)